In [ ]:
import os
project_path = os.getcwd()
project_path

'/fs/scratch/PDE0005/projects/telescope_TCC00573'

In [ ]:
# read data files (fastq files)
rawdata_path = "/fs/scratch/PDE0005/projects/telescope-pipeline-demo/fastq"

In [4]:
import os
import pandas as pd
from pathlib import Path
import shutil
from shutil import rmtree

In [5]:
def delete_file_or_dir(pth: str) -> None:
    """
    Deletes a file or directory
    
    Arguments:
    -----------
    - path: A file or directory path. It could either be relative or absolute.
    """
    pth = Path(pth)
    for child in pth.glob('*'):
        if child.is_file():
            child.unlink()
        else:
            rmtree(child)
    pth.rmdir()   
                

def create_dir(pth: str):
    """
    Create a directory
    
    Arguments:
    -----------
    - path: A directory path. It could either be relative or absolute.
    """
    try:
        Path(pth).mkdir(parents=True, exist_ok=True)
    except OSError as e:
        raise e

In [ ]:
# create output directory
!mkdir -p output/BOWTIE
!mkdir -p output/TELESCOPE

In [ ]:
# create manifest and jobs scrpts directories
manifests_dpath: str = "manifests"
jobs_s1_dpath: str = "jobs_step1"
jobs_s2_dpath: str = "jobs_step2"

In [5]:
if os.path.isdir(manifests_dpath):
    delete_file_or_dir(manifests_dpath)
create_dir(manifests_dpath)

In [8]:
if os.path.isdir(jobs_s1_dpath):
    delete_file_or_dir(jobs_s1_dpath)
create_dir(jobs_s1_dpath)

if os.path.isdir(jobs_s2_dpath):
    delete_file_or_dir(jobs_s2_dpath)
create_dir(jobs_s2_dpath)

In [13]:
def read_fastq_files(rawdata_path: str) -> list:
    """
    Read FASTQ files from the specified path and return unique sample identifiers.

    Supported filename patterns:
      - <sample>_R1.fastq.gz / <sample>_R2.fastq.gz
      - <sample>_1.fastq.gz  / <sample>_2.fastq.gz

    Arguments:
    ----------
    rawdata_path : str
        Path to directory containing FASTQ files

    Returns:
    --------
    list
        Sorted list of unique sample identifiers
    """
    import os
    import glob
    import re

    pattern = os.path.join(rawdata_path, "*.fastq.gz")
    fastq_files = glob.glob(pattern)

    sample_ids = set()

    # Regex: strip _R1/_R2 or _1/_2 before .fastq.gz
    regex = re.compile(r"(.+?)(?:_R?[12])\.fastq\.gz$")

    for file_path in fastq_files:
        filename = os.path.basename(file_path)
        match = regex.match(filename)
        if match:
            sample_ids.add(match.group(1))

    return sorted(sample_ids)


In [ ]:
# Test the function
sample_ids = read_fastq_files(rawdata_path)
print(f"Found {len(sample_ids)} unique samples:")
for sample_id in sample_ids:
    print(f"  {sample_id}")


In [15]:
IGNORE_SAMPLE_IDS = [
    "SL259668",
    "SL349334",
]

In [16]:
def generate_manifest_files(
    sample_ids: list,
    rawdata_path: str,
    manifests_dpath: str,
    samples_per_manifest: int = 4
) -> list:
    """
    Generate manifest TSV files from sample IDs.

    Supported FASTQ naming conventions:
      - <sample>_R1.fastq.gz / <sample>_R2.fastq.gz
      - <sample>_1.fastq.gz  / <sample>_2.fastq.gz
    """
    import os
    import math

    os.makedirs(manifests_dpath, exist_ok=True)

    def resolve_fastq_pair(sample_id: str):
        """Return (fq1, fq2) paths for a sample, or raise if not found."""
        candidates = [
            (f"{sample_id}_R1.fastq.gz", f"{sample_id}_R2.fastq.gz"),
            (f"{sample_id}_1.fastq.gz",  f"{sample_id}_2.fastq.gz"),
        ]

        for fq1, fq2 in candidates:
            fq1_path = os.path.join(rawdata_path, fq1)
            fq2_path = os.path.join(rawdata_path, fq2)
            if os.path.isfile(fq1_path) and os.path.isfile(fq2_path):
                return fq1_path, fq2_path

        if not sample_id in IGNORE_SAMPLE_IDS:
            raise FileNotFoundError(
                f"No valid FASTQ pair found for sample '{sample_id}' "
                f"in {rawdata_path}"
            )
        else:
            return None

    # Calculate number of manifest files needed
    num_manifests = math.ceil(len(sample_ids) / samples_per_manifest)
    manifest_files = []

    for i in range(num_manifests):
        start_idx = i * samples_per_manifest
        end_idx = min(start_idx + samples_per_manifest, len(sample_ids))
        manifest_samples = sample_ids[start_idx:end_idx]

        lines = ["sample\tfq1\tfq2"]

        for sample_id in manifest_samples:
            res = resolve_fastq_pair(sample_id)
            if res is None:
                continue
            fq1_path, fq2_path = res
            lines.append(f"{sample_id}\t{fq1_path}\t{fq2_path}")

        manifest_filename = f"manifest_{i}.tsv"
        manifest_filepath = os.path.join(manifests_dpath, manifest_filename)

        with open(manifest_filepath, "w") as f:
            f.write("\n".join(lines) + "\n")

        manifest_files.append(manifest_filepath)
        print(f"Generated {manifest_filename} with {len(manifest_samples)} samples")

    return manifest_files


In [ ]:
# Generate manifest files
manifest_files = generate_manifest_files(sample_ids, rawdata_path, manifests_dpath)

print(f"\nGenerated {len(manifest_files)} manifest files:")
for manifest_file in manifest_files:
    print(f"  {manifest_file}")

# Display content of first manifest file as example
if manifest_files:
    print(f"\nContent of {os.path.basename(manifest_files[0])}:")
    with open(manifest_files[0], 'r') as f:
        print(f.read())


In [ ]:
bowtie_job_template = """#!/bin/bash
#SBATCH --time=96:00:00
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=[NO_CORES]
#SBATCH --mem=256G
#SBATCH --job-name=immuno_retrovirus
#SBATCH --account=PDE0005
#SBATCH --error=immuno_retrovirus-%j.err
#SBATCH --output=immuno_retrovirus-%j.out

set -euo pipefail
set -x

cd "${SLURM_SUBMIT_DIR}"

module load miniconda3/24.1.2-py310
source "$(conda info --base)/etc/profile.d/conda.sh"
conda activate immuno_telescope

export PROJECT_PATH="[PROJECT_FULL_PATH]"
export SAMPLES_PATH="[SAMPLE_FULL_PATH]"
NODE_TMPDIR="${SLURM_TMPDIR:-/tmp}"
REF_DIR="/fs/scratch/PDE0005/telescope_lungsamples/data/REF"
INDEX_DIR="/fs/scratch/PDE0005/telescope_lungsamples/data/Indexes"

rm -rf "${PROJECT_PATH}/tmp_[MANIFEST_ID]"
mkdir -p "${PROJECT_PATH}/tmp_[MANIFEST_ID]"
mkdir -p "${PROJECT_PATH}/output/job_[MANIFEST_ID]"

# copy code + manifests into node-local space
cp -R \
  "/fs/scratch/PDE0005/telescope_lungsamples/code/lib" \
  "/fs/scratch/PDE0005/telescope_lungsamples/code/run.py" \
  "${PROJECT_PATH}/manifests/manifest_[MANIFEST_ID].tsv" \
  "${NODE_TMPDIR}/"

cd "${NODE_TMPDIR}"

python run.py \
  --manifest manifest_[MANIFEST_ID].tsv \
  --gtf "${REF_DIR}/gencode.v39.annotation.gtf" \
  --genome "${REF_DIR}/GRCh38.p13.genome.fa" \
  --transcript "${REF_DIR}/gencode.v39.transcripts.fa" \
  --herv_gtf "${REF_DIR}/HG38_HERV_LINE_all_families_telescope_ann.gtf" \
  --bowtiew2_idx "${INDEX_DIR}/gencode.v39_bowtie2/human" \
  --workflows BOWTIE \
  --out_dir "${PROJECT_PATH}/output/job_[MANIFEST_ID]" \
  --samples_dir "${SAMPLES_PATH}" \
  --n_cores "${SLURM_CPUS_PER_TASK}" \
  --trimgalore_n_cores 6 \
  --telescope_n_cores 1 \
  --seed 123456 \
  --scratch_dir "${PROJECT_PATH}/tmp_[MANIFEST_ID]" \
  --fastq_mode \
  --sample_check_n_cores 6

LOGS="${PROJECT_PATH}/output/job_[MANIFEST_ID]/logs.tsv"
if [ -f "${LOGS}" ]; then
  mkdir -p "${PROJECT_PATH}/output/BOWTIE"
  cp "${LOGS}" "${PROJECT_PATH}/output/BOWTIE/logs_[MANIFEST_ID].tsv"
fi

BOWTIE_DIR="${PROJECT_PATH}/output/job_[MANIFEST_ID]/BOWTIE"
if [ -d "${BOWTIE_DIR}" ]; then
  mkdir -p "${PROJECT_PATH}/output/BOWTIE"
  rsync -a "${BOWTIE_DIR}/" "${PROJECT_PATH}/output/BOWTIE/"
fi


"""

In [ ]:
telescope_job_template: str = """#!/bin/bash
#SBATCH --time=144:00:00
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=[NO_CORES]
#SBATCH --mem=256GB
#SBATCH --job-name=immuno_retrovirus
#SBATCH --account=PDE0005
#SBATCH -e immuno_retrovirus-%j.err
#SBATCH -o immuno_retrovirus-%j.out

set echo

# slurm starts job in working DIR
cd $SLURM_SUBMIT_DIR

# set up software environment
module load intel/2021.10.0
module load miniconda3/24.1.2-py310

source activate immuno_telescope

export CMPLR_ROOT="/apps/spack/0.21/ascend/linux-rhel9-zen2/intel-oneapi-compilers/gcc/11.4.1/2023.2.3-nkzjnam/compiler/2023.2.3"
export INTEL64="$CMPLR_ROOT/linux/compiler/lib/intel64_lin"
export LD_LIBRARY_PATH="$INTEL64:$CMPLR_ROOT/linux/lib:$CMPLR_ROOT/linux/lib/x64:$CONDA_PREFIX/lib:${LD_LIBRARY_PATH}"
# Optional (only if you still see unresolved Intel symbols):
export LD_PRELOAD="$INTEL64/libimf.so:$INTEL64/libsvml.so:$INTEL64/libirc.so"

# 4) (Optional) sanity check that the Telescope C extension loads
python - <<'PY'
import ctypes, glob, os, sys
pats = glob.glob(os.path.expanduser("~/.conda/envs/immuno_telescope/lib/python3.6/site-packages/telescope/utils/calignment*.so"))
if not pats:
    sys.exit("Telescope extension not found")
ctypes.CDLL(pats[0])
print("Loaded Telescope extension OK:", pats[0])
PY

export PROJECT_PATH="[PROJECT_FULL_PATH]"
export SAMPLES_PATH="[SAMPLE_FULL_PATH]"
NODE_TMPDIR="${SLURM_TMPDIR:-/tmp}"
REF_DIR="/fs/scratch/PDE0005/telescope_lungsamples/data/REF"
INDEX_DIR="/fs/scratch/PDE0005/telescope_lungsamples/data/Indexes"

mkdir -p "${PROJECT_PATH}/output/job_[MANIFEST_ID]"

# copy data to compute node local space ${NODE_TMPDIR}
cp -R \
    /fs/scratch/PDE0005/telescope_lungsamples/code/lib \
    /fs/scratch/PDE0005/telescope_lungsamples/code/run.py \
    ${NODE_TMPDIR}/

cd ${NODE_TMPDIR}

# $PFSDIR: /fs/scratch/PAS0438/osu9787/18091549.owens

    
python run.py \
    --gtf "${REF_DIR}/gencode.v39.annotation.gtf" \
    --genome "${REF_DIR}/GRCh38.p13.genome.fa" \
    --transcript "${REF_DIR}/gencode.v39.transcripts.fa" \
    --herv_gtf "${REF_DIR}/HG38_HERV_LINE_all_families_telescope_ann.gtf" \
    --bowtiew2_idx "${INDEX_DIR}/gencode.v39_bowtie2/human" \
    --workflows TELESCOPE \
    --out_dir "${PROJECT_PATH}/output/job_[MANIFEST_ID]" \
    --scratch_dir $PROJECT_PATH/tmp_[MANIFEST_ID] \
    --n_cores [NO_CORES] \
    --trimgalore_n_cores 6 \
    --telescope_n_cores 1 \
    --seed 123456 \
    --fastq_mode

LOGS="${PROJECT_PATH}/output/job_[MANIFEST_ID]/logs.tsv"
if [ -f "${LOGS}" ]; then
  mkdir -p "${PROJECT_PATH}/output/TELESCOPE"
  cp "${LOGS}" "${PROJECT_PATH}/output/TELESCOPE/logs_[MANIFEST_ID].tsv"
fi

TELESCOPE_DIR="${PROJECT_PATH}/output/job_[MANIFEST_ID]/TELESCOPE"
if [ -d "${TELESCOPE_DIR}" ]; then
  mkdir -p "${PROJECT_PATH}/output/TELESCOPE"
  rsync -a "${TELESCOPE_DIR}/" "${PROJECT_PATH}/output/TELESCOPE/"
fi


"""

In [ ]:
num_manifests: int = len(manifest_files)
for i in range(num_manifests):
    j_tmp: str = bowtie_job_template
        
    with open(Path(jobs_s1_dpath, f"job_{i}.sh"), "w") as f:
        f.write(j_tmp.replace("[NO_CORES]", "24").replace("[MANIFEST_ID]", str(i)))
        f.write(j_tmp.replace("[PROJECT_FULL_PATH]", project_path).replace("[SAMPLE_FULL_PATH]", rawdata_path))
    
    print(f"sbatch {jobs_s1_dpath}/job_{i}.sh --gres=pfsdir:ess")

In [ ]:
num_manifests: int = len(manifest_files)
for i in range(num_manifests):
    j_tmp: str = telescope_job_template
        
    with open(Path(jobs_s2_dpath, f"job_{i}.sh"), "w") as f:
        f.write(j_tmp.replace("[NO_CORES]", "24").replace("[MANIFEST_ID]", str(i)))
        f.write(j_tmp.replace("[PROJECT_FULL_PATH]", project_path).replace("[SAMPLE_FULL_PATH]", rawdata_path))
    
    print(f"sbatch {jobs_s2_dpath}/job_{i}.sh --gres=pfsdir:ess")
    